## Assessing Discussions of LLM Limitations: Multi-Prompt Evaluation with GPT 
This script employs five unique prompting techniques, including zero-shot and few-shot approaches, to guide GPT-4.o in evaluating the extent of LLM limitations discussed in academic papers. It aims to precisely rate each paper based on the thoroughness of its discussion on LLM limitations, using targeted prompts to achieve an in-depth analysis.

### Prompt1: Standard Baseline Evaluation Prompt

In [ ]:
import openai
import json
import csv
from tqdm import tqdm

openai.api_key = 'Place your API Key here'

def evaluate_paper_baseline(title, summary):
    prompt = f"""
    Can you please let me know whether the following paper is about language models (e.g., LMs or LLMs) and whether it talks about their limitations? If so, please indicate the parts in the abstract or title it does so. Please be brief in your explanations. Note that LMs and LLMs include pre-trained transformer based language models and multimodal, visual language models. Please include *all* kinds of language models but *no* other, more general models in your classification.

    Answer in the following way:
    LMs: [yes/no].
    Limitations of LMs: [rating from 1-5].
    Evidence: [the evidence text in the abstract or title].

    Title: {title}
    Paper: {summary}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5
    )

    return response.choices[0].message['content'].strip()

def read_papers_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        papers = json.load(file)
    return papers

file_path = '10_gold_standard_papers.json'

csv_file_path = 'baseline_10_results.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'LMs', 'Limitations of LMs', 'Evidence'])

    papers = read_papers_from_json(file_path)

    for paper in tqdm(papers, desc="Processing papers"):
        title = paper.get('title', 'No Title')
        summary = paper.get('summary', 'No Summary')
        evaluation_result = evaluate_paper_baseline(title, summary)
        print(f"Raw evaluation result:\n{evaluation_result}\n")

        lines = evaluation_result.split('\n')
        talks_about_llms = ''
        rate = ''
        evidence = ''

        for line in lines:
            line = line.strip()
            if line.startswith('LMs:'):
                talks_about_llms = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Limitations of LMs:'):
                rate = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Evidence:'):
                evidence = line.split(':', 1)[1].strip().strip('[]')

        writer.writerow([title, talks_about_llms, rate, evidence])
        print(f"Title: {title}\nEvaluation Result: {evaluation_result}\n")

        print(f"Parsed values:\nTitle: {title}\nLMs: {talks_about_llms}\nLimitations of LMs: {rate}\nEvidence: {evidence}\n")

### Prompt2: Enhanced Instructional Detail Prompt

In [1]:
import openai
import json
import csv
from tqdm import tqdm

openai.api_key = ' Enter your API key here'

def evaluate_paper_baseline(title, summary):
    prompt = f"""
    Can you please let me know whether the following paper is about language models (e.g., LMs or LLMs) and whether it talks about their limitations? If so, please indicate the parts in the abstract or title it does so. Please be brief in your evidence. Note that LMs and LLMs include pre-trained transformer based language models and multimodal, visual language models. Please include *all* kinds of language models but *no* other, more general models in your classification.

    Based on the following rules, rate the abstract from 1-5 based on how thoroughly it discusses the limitations or challenges related to LLMs:
    1: Does not talk about LLMs at all or mention any limitation of LLMs.
    2: Mentions one limitation of LLMs very briefly.
    3: Mentions limitations of LLMs, but they are not the focus of the abstract.The limitations are discussed superficially or as secondary points.
    4: Provides multiple limitations of LLMs. The limitations are significant and discussed in details but alongside other topics.
    5: The entire abstract or most of the sentences focus on the limitations and challenges of LLMs. Sentences discuss limitations in detail, with strong wording indicating serious issues.


    Please answer in the following format by providing the rating and a brief evidence for each abstract.

    Answer in the following way:
    Does it talk about LLMs: [yes/no].
    Rate Limitations of LLMs: [rating from 1-5].
    Evidence: [the evidence text in the abstract or title].

    Title: {title}
    Paper: {summary}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5
    )

    return response.choices[0].message['content'].strip()

def read_papers_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        papers = json.load(file)
    return papers

file_path = '10_gold_standard_papers.json'

csv_file_path = 'my_prompt_10_results.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Talks about LLMs', 'Rate', 'Evidence'])

    papers = read_papers_from_json(file_path)

    for paper in tqdm(papers, desc="Processing papers"):
        title = paper.get('title', 'No Title')
        summary = paper.get('summary', 'No Summary')
        evaluation_result = evaluate_paper_baseline(title, summary)

        # parsing the response
        lines = evaluation_result.split('\n')
        talks_about_llms = ''
        rate = ''
        evidence = ''
        
        for line in lines:
            if line.startswith('Does it talk about LLMs:'):
                talks_about_llms = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Rate Limitations of LLMs:'):
                rate = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Evidence:'):
                evidence = line.split(':')[1].strip().strip('[]')

        # results written on csv file 
        writer.writerow([title, talks_about_llms, rate, evidence])
        print(f"Title: {title}\nEvaluation Result: {evaluation_result}\n")


Processing papers:   3%|██▏                                                             | 1/29 [00:01<00:55,  1.97s/it]

Title: Directed Acyclic Transformer Pre-training for High-quality Non-autoregressive Text Generation
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 3.
Evidence: "existing NAR models lack proper pre-training, making them still far behind the pre-trained autoregressive models" and "alleviates the error accumulation problem in autoregressive generation."



Processing papers:   7%|████▍                                                           | 2/29 [00:02<00:36,  1.36s/it]

Title: Time-and-Space-Efficient Weighted Deduction
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract and title discuss weighted deduction in NLP algorithms, topological sorting, and dynamic programming but do not mention language models or their limitations.



Processing papers:  10%|██████▌                                                         | 3/29 [00:04<00:37,  1.43s/it]

...

Title: Multi 3 WOZ: A Multilingual, Multi-Domain, Multi-Parallel Dataset for Training and Evaluating Culturally Adapted Task-Oriented Dialog Systems
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract discusses the creation and evaluation of a multilingual, multi-domain, and multi-parallel dataset for task-oriented dialog systems. It focuses on the challenges of creating culturally adapted datasets and does not mention language models (LMs or LLMs) or their limitations.



Processing papers:  97%|████████████████████████████████████████████████████████████▊  | 28/29 [00:46<00:01,  1.95s/it]

Title: Can Authorship Representation Learning Capture Stylistic Features?
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract discusses authorship representation learning and its ability to capture stylistic features, but it does not mention language models (LMs or LLMs) or any limitations associated with them. The focus is on authorship attribution and writing style.



Processing papers: 100%|███████████████████████████████████████████████████████████████| 29/29 [00:47<00:00,  1.62s/it]

Title: Optimal Transport Posterior Alignment for Cross-lingual Semantic Parsing
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract focuses on cross-lingual semantic parsing and methods to improve it using Optimal Transport. It does not mention language models (LMs or LLMs) or discuss their limitations.



### Prompt3: Categorical Instruction Grouping Prompt

In [8]:
import openai
import json
import csv
from tqdm import tqdm

openai.api_key = 'Enter your API key here'

def evaluate_paper_baseline(title, summary):
    prompt = f"""
    Please evaluate the following paper to determine if it discusses language models (e.g., LMs or LLMs) and whether it addresses their limitations. If it does, indicate the relevant parts of the abstract or title. Note that LMs and LLMs include pre-trained transformer-based language models and multimodal, visual language models. Include all kinds of language models but exclude other, more general models.

    Please rate the papers from 1 to 5:
    - **1:** The abstract does not talk about large language models at all, or even if it talks about 
    LLMs, it does not mention any limitation of them.
    - **2-3:** The abstract mentions just a few limitations of Large Language Models; they are 
    mentioned as secondary points.
    - **4-5:** The abstract explicitly talks a lot about the limitations of Large Language Models 
    and discusses them in detail or it uses strong wording.

    Please answer in the following format by providing the rating and a brief evidence for 
    each abstract.

    Answer in the following way:
    Does it talk about LLMs: [yes/no].
    Rate Limitations of LLMs: [rating from 1-5].
    Evidence: [the evidence text in the abstract or title].

    Title: {title}
    Paper: {summary}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5
    )

    return response.choices[0].message['content'].strip()

def read_papers_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        papers = json.load(file)
    return papers

file_path = '10_gold_standard_papers.json'

csv_file_path = 'my_simpler_prompt_10_results.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Talks about LLMs', 'Rate', 'Evidence'])

    papers = read_papers_from_json(file_path)

    for paper in tqdm(papers, desc="Processing papers"):
        title = paper.get('title', 'No Title')
        summary = paper.get('summary', 'No Summary')
        evaluation_result = evaluate_paper_baseline(title, summary)

        # parsing the response
        lines = evaluation_result.split('\n')
        talks_about_llms = ''
        rate = ''
        evidence = ''
        
        for line in lines:
            if line.startswith('Does it talk about LLMs:'):
                talks_about_llms = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Rate Limitations of LLMs:'):
                rate = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Evidence:'):
                evidence = line.split(':')[1].strip().strip('[]')

        # results written on csv file 
        writer.writerow([title, talks_about_llms, rate, evidence])
        print(f"Title: {title}\nEvaluation Result: {evaluation_result}\n")


Processing papers:   3%|██▏                                                             | 1/29 [00:02<01:00,  2.15s/it]

Title: Directed Acyclic Transformer Pre-training for High-quality Non-autoregressive Text Generation
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 2.
Evidence: "existing NAR models lack proper pre-training, making them still far behind the pre-trained autoregressive models" and "Further analysis shows that PreDAT benefits from the unbiased prediction order that alleviates the error accumulation problem in autoregressive generation."

The abstract mentions limitations of existing NAR models compared to pre-trained autoregressive models and discusses the error accumulation problem in autoregressive generation. However



Processing papers:   7%|████▍                                                           | 2/29 [00:03<00:45,  1.69s/it]

Title: Time-and-Space-Efficient Weighted Deduction
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract does not mention large language models or their limitations. It focuses on weighted deduction strategies in NLP algorithms and their efficiency in terms of time and space.



Processing papers:  10%|██████▌                                                         | 3/29 [00:04<00:39,  1.52s/it]

...

Title: Multi 3 WOZ: A Multilingual, Multi-Domain, Multi-Parallel Dataset for Training and Evaluating Culturally Adapted Task-Oriented Dialog Systems
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract does not mention large language models or their limitations. It focuses on the creation and evaluation of a multilingual, multi-domain, multi-parallel task-oriented dialog dataset.



Processing papers:  97%|████████████████████████████████████████████████████████████▊  | 28/29 [00:39<00:01,  1.24s/it]

Title: Can Authorship Representation Learning Capture Stylistic Features?
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract focuses on authorship representation learning and its ability to capture stylistic features. It does not mention or discuss large language models (LLMs) or their limitations.



Processing papers: 100%|███████████████████████████████████████████████████████████████| 29/29 [00:40<00:00,  1.41s/it]

Title: Optimal Transport Posterior Alignment for Cross-lingual Semantic Parsing
Evaluation Result: Does it talk about LLMs: no.
Rate Limitations of LLMs: 1.
Evidence: The abstract focuses on cross-lingual semantic parsing and methods to improve performance using Optimal Transport. It does not mention large language models or their limitations.



### Prompt4: Selective Few-Shot Prompting Technique

In [22]:
import openai
import json
import csv
from tqdm import tqdm

openai.api_key = 'Enter API key here'

def evaluate_paper_few_shot(title, summary):
    few_shot_examples = """
    ### Example Evaluations:

    **Example 1:**
    Title: SPAE Semantic Pyramid AutoEncoder for Multimodal Generation with Frozen LLMs
    Paper: "In this work, we introduce Semantic Pyramid AutoEncoder (SPAE) for enabling frozen LLMs to perform both understanding and generation tasks involving non-linguistic modalities such as images or videos. SPAE converts between raw pixels and interpretable lexical tokens (or words) extracted from the LLM's vocabulary. The resulting tokens capture both the semantic meaning and the fine-grained details needed for visual reconstruction, effectively translating the visual content into a language comprehensible to the LLM, and empowering it to perform a wide array of multimodal tasks. Our approach is validated through in-context learning experiments with frozen PaLM 2 and GPT 3.5 on a diverse set of image understanding and generation tasks. Our method marks the first successful attempt to enable a frozen LLM to generate image content while surpassing state-of-the-art performance in image understanding tasks, under the same setting, by over 25%"
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 1.
    Evidence: "In this work, we introduce Semantic Pyramid AutoEncoder (SPAE) for enabling frozen LLMs to perform both understanding and generation tasks involving non-linguistic modalities such as images or videos."

    **Example 2:**
    Title: Large Language Models for Conducting Advanced Text Analytics Information Systems Research
    Paper: "The exponential growth of digital content has generated massive textual datasets, necessitating advanced analytical approaches. Large Language Models (LLMs) have emerged as tools capable of processing and extracting insights from massive unstructured textual datasets. However, how to leverage LLMs for text-based Information Systems (IS) research is currently unclear. To assist IS research in understanding how to operationalize LLMs, we propose a Text Analytics for Information Systems Research (TAISR) framework. Our proposed framework provides detailed recommendations grounded in IS and LLM literature on how to conduct meaningful text-based IS research. We conducted three case studies in business intelligence using our TAISR framework to demonstrate its application across several IS research contexts. We also outline potential challenges and limitations in adopting LLMs for IS. By offering a systematic approach and evidence of its utility, our TAISR framework contributes to future IS research streams looking to incorporate powerful LLMs for text analytics."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 2.
    Evidence: "We also outline potential challenges and limitations in adopting LLMs for IS."
    
    **Example 3:**
    Title: Meta-Reasoning: Semantics-Symbol Deconstruction for Large Language Models
    Paper: "Neural-symbolic methods have demonstrated efficiency in enhancing the reasoning abilities of large language models (LLMs). However, existing methods mainly rely on syntactically mapping natural languages to complete formal languages like Python and SQL. Those methods require that reasoning tasks be convertible into programs, which cater to the computer execution mindset and deviate from human reasoning habits. To broaden symbolic methods' applicability and adaptability in the real world, we propose the Meta-Reasoning from a linguistic perspective. This method empowers LLMs to deconstruct reasoning-independent semantic information into generic symbolic representations, thereby efficiently capturing more generalized reasoning knowledge. We conduct extensive experiments on more than ten datasets encompassing conventional reasoning tasks like arithmetic, symbolic, and logical reasoning, and the more complex interactive reasoning tasks like theory-of-mind reasoning. Experimental results demonstrate that Meta-Reasoning significantly enhances in-context reasoning accuracy, learning efficiency, out-of-domain generalization, and output stability compared to the Chain-of-Thought technique. Code and data are publicly available at \\url{https://github.com/Alsace08/Meta-Reasoning}."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 3.
    Evidence: "existing methods mainly rely on syntactically mapping natural languages to complete formal languages like Python and SQL. Those methods require that reasoning tasks be convertible into programs, which cater to the computer execution mindset and deviate from human reasoning habits."
   
    **Example 4:**
    Title: Fairness in Large Language Models: A Taxonomic Survey
    Paper: "Large Language Models (LLMs) have demonstrated remarkable success across various domains. However, despite their promising performance in numerous real-world applications, most of these algorithms lack fairness considerations. Consequently, they may lead to discriminatory outcomes against certain communities, particularly marginalized populations, prompting extensive study in fair LLMs. On the other hand, fairness in LLMs, in contrast to fairness in traditional machine learning, entails exclusive backgrounds, taxonomies, and fulfillment techniques. To this end, this survey presents a comprehensive overview of recent advances in the existing literature concerning fair LLMs. Specifically, a brief introduction to LLMs is provided, followed by an analysis of factors contributing to bias in LLMs. Additionally, the concept of fairness in LLMs is discussed categorically, summarizing metrics for evaluating bias in LLMs and existing algorithms for promoting fairness. Furthermore, resources for evaluating bias in LLMs, including toolkits and datasets, are summarized. Finally, existing research challenges and open questions are discussed."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 4.
    Evidence: "most of these algorithms lack fairness considerations. Consequently, they may lead to discriminatory outcomes against certain communities, particularly marginalized populations," and "an analysis of factors contributing to bias in LLMs. Additionally, the concept of fairness in LLMs is discussed categorically, summarizing metrics for evaluating bias in LLMs and existing algorithms for promoting fairness. Furthermore, resources for evaluating bias in LLMs, including toolkits and datasets, are summarized."
    
    **Example 5:**
    Title: Lost in the Middle: How Language Models Use Long Contexts
    Paper: "While recent language models have the ability to take long contexts as input, relatively little is known about how well they use longer context. We analyze the performance of language models on two tasks that require identifying relevant information in their input contexts: multi-document question answering and key-value retrieval. We find that performance can degrade significantly when changing the position of relevant information, indicating that current language models do not robustly make use of information in long input contexts. In particular, we observe that performance is often highest when relevant information occurs at the beginning or end of the input context, and significantly degrades when models must access relevant information in the middle of long contexts, even for explicitly long-context models. Our analysis provides a better understanding of how language models use their input context and provides new evaluation protocols for future long-context language models."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 5.
    Evidence: "We find that performance can degrade significantly when changing the position of relevant information, indicating that current language models do not robustly make use of information in long input contexts."
    
    """
    
    prompt = f"""
    Please evaluate the following paper to determine if it discusses language models (e.g., LMs or LLMs) and whether it addresses their limitations. If it does, indicate the relevant parts of the abstract or title. Note that LMs and LLMs include pre-trained transformer-based language models and multimodal, visual language models. Include all kinds of language models but exclude other, more general models.

    Please rate the papers from 1 to 5:
    - **1:** The abstract does not talk about large language models at all, or even if it talks about LLMs, it does not mention any limitation of them.
    - **2-3:** The abstract mentions just a few limitations of Large Language Models; they are mentioned as secondary points.
    - **4-5:** The abstract explicitly talks a lot about the limitations of Large Language Models and discusses them in detail or it uses strong wording.

    {few_shot_examples}
    
    Please answer in the following format by providing the rating and a brief evidence for each abstract.

    Answer in the following way:
    Does it talk about LLMs: [yes/no].
    Rate Limitations of LLMs: [1-5].
    Evidence: [the evidence text in the abstract or title].

    Title: {title}
    Paper: {summary}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-2024-05-13",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5
    )

    return response.choices[0]['message']['content'].strip()

def read_papers_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        papers = json.load(file)
    return papers

file_path = '10_gold_standard_papers.json'

csv_file_path = 'few_shot_prompt_10_results.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Talks about LLMs', 'Rate', 'Evidence'])

    papers = read_papers_from_json(file_path)

    for paper in tqdm(papers, desc="Processing papers"):
        title = paper.get('title', 'No Title')
        summary = paper.get('summary', 'No Summary')
        evaluation_result = evaluate_paper_few_shot(title, summary)

        # parsing the response
        lines = evaluation_result.split('\n')
        talks_about_llms = ''
        rate = ''
        evidence = ''
        
        for line in lines:
            if line.startswith('Does it talk about LLMs:'):
                talks_about_llms = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Rate Limitations of LLMs:'):
                rate = line.split(':')[1].strip().strip('[]')
            elif line.startswith('Evidence:'):
                evidence = line.split(':')[1].strip().strip('[]')

        # results written on csv file 
        writer.writerow([title, talks_about_llms, rate, evidence])
        print(f"Title: {title}\nEvaluation Result: {evaluation_result}\n")



Processing papers:   3%|██▏                                                             | 1/29 [00:01<00:46,  1.66s/it]

Title: Directed Acyclic Transformer Pre-training for High-quality Non-autoregressive Text Generation
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract focuses on Non-AutoRegressive (NAR) text generation models and their pre-training, without mentioning large language models or their limitations.



Processing papers:   7%|████▍                                                           | 2/29 [00:05<01:12,  2.68s/it]

Title: Time-and-Space-Efficient Weighted Deduction
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract discusses weighted deduction strategies and their efficiency in terms of time and space. There is no mention of language models, large language models, or their limitations.



Processing papers:  10%|██████▌                                                         | 3/29 [00:07<01:05,  2.53s/it]

...

Title: How Abstract Is Linguistic Generalization in Large Language Models? Experiments with Argument Structure
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 5.
Evidence: "However, LLMs fail at generalizations between related contexts that have not been observed during pre-training, but which instantiate more abstract, but well-attested structural generalizations (e.g., between the active object and passive subject of an arbitrary verb). Instead, in this case, LLMs show a bias to generalize based on linear order. This finding points to a limitation



Processing papers:  93%|██████████████████████████████████████████████████████████▋    | 27/29 [00:53<00:04,  2.28s/it]

Title: Multi 3 WOZ: A Multilingual, Multi-Domain, Multi-Parallel Dataset for Training and Evaluating Culturally Adapted Task-Oriented Dialog Systems
Evaluation Result: Does it talk about LLMs: No.
Rate Limitations of LLMs: 1.
Evidence: The abstract discusses the creation and evaluation of a multilingual, multi-domain, multi-parallel dataset for task-oriented dialog systems. It does not mention large language models (LLMs) or their limitations.



Processing papers:  93%|██████████████████████████████████████████████████████████▋    | 27/29 [00:53<00:03,  1.97s/it]


RateLimitError: Rate limit reached for gpt-4o in organization org-cIU00Ks8lIudySaVYr0LQOp3 on tokens per min (TPM): Limit 30000, Used 27740, Requested 2704. Please try again in 888ms. Visit https://platform.openai.com/account/rate-limits to learn more.

### Prompt5: Detailed Explanatory Few-Shot Prompting

In [5]:
import openai
import json
import csv
import os
from tqdm import tqdm
import time

#openai.api_key = 'Enter your API key here' #my api 
openai.api_key = 'Enter your API key here' #---> 2nd API
# Keep track of progress with an index
progress_file = 'progress_few_shot.txt'

def evaluate_paper_few_shot(title, summary):
    prompt = f"""
    Please evaluate the following paper to determine if it discusses language models (e.g., LMs or LLMs) and whether it addresses their limitations. If it does, indicate the relevant parts of the abstract or title. Note that LMs and LLMs include pre-trained transformer-based language models and multimodal, visual language models. Include all kinds of language models but exclude other, more general models.
    Please look at the following examples alongside the explanations on why decided the respective ratings and rate the other abstracts from 1 to 5 accordingly by following the same logic as below: 
    
    **Example Output 1:**
    Title: SPAE Semantic Pyramid AutoEncoder for Multimodal Generation with Frozen LLMs
    Paper: "In this work, we introduce Semantic Pyramid AutoEncoder (SPAE) for enabling frozen LLMs to perform both understanding and generation tasks involving non-linguistic modalities such as images or videos. SPAE converts between raw pixels and interpretable lexical tokens (or words) extracted from the LLM's vocabulary. The resulting tokens capture both the semantic meaning and the fine-grained details needed for visual reconstruction, effectively translating the visual content into a language comprehensible to the LLM, and empowering it to perform a wide array of multimodal tasks. Our approach is validated through in-context learning experiments with frozen PaLM 2 and GPT 3.5 on a diverse set of image understanding and generation tasks. Our method marks the first successful attempt to enable a frozen LLM to generate image content while surpassing state-of-the-art performance in image understanding tasks, under the same setting, by over 25%"
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 1.
    Evidence: "In this work, we introduce Semantic Pyramid AutoEncoder (SPAE) for enabling frozen LLMs to perform both understanding and generation tasks involving non-linguistic modalities such as images or videos."
    
    Output Explanation: This paper should be rated with 1 since even though it talks about LLMs, it does not mention any explicit limitation of the models in the abstract. Note: Additionally, papers that do not talk about LLMs at all, rate them with 1. 
    
    **Example Output 2:**
    Title: Large Language Models for Conducting Advanced Text Analytics Information Systems Research
    Paper: "The exponential growth of digital content has generated massive textual datasets, necessitating advanced analytical approaches. Large Language Models (LLMs) have emerged as tools capable of processing and extracting insights from massive unstructured textual datasets. However, how to leverage LLMs for text-based Information Systems (IS) research is currently unclear. To assist IS research in understanding how to operationalize LLMs, we propose a Text Analytics for Information Systems Research (TAISR) framework. Our proposed framework provides detailed recommendations grounded in IS and LLM literature on how to conduct meaningful text-based IS research. We conducted three case studies in business intelligence using our TAISR framework to demonstrate its application across several IS research contexts. We also outline potential challenges and limitations in adopting LLMs for IS. By offering a systematic approach and evidence of its utility, our TAISR framework contributes to future IS research streams looking to incorporate powerful LLMs for text analytics."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 2.
    Evidence: "We also outline potential challenges and limitations in adopting LLMs for IS."
    
    Output Explanation: This abstract mentions just one limitation of the Large Language Models and focuses on other topics.
    
    **Example Output 3:**
    Title: Meta-Reasoning: Semantics-Symbol Deconstruction for Large Language Models
    Paper: "Neural-symbolic methods have demonstrated efficiency in enhancing the reasoning abilities of large language models (LLMs). However, existing methods mainly rely on syntactically mapping natural languages to complete formal languages like Python and SQL. Those methods require that reasoning tasks be convertible into programs, which cater to the computer execution mindset and deviate from human reasoning habits. To broaden symbolic methods' applicability and adaptability in the real world, we propose the Meta-Reasoning from a linguistic perspective. This method empowers LLMs to deconstruct reasoning-independent semantic information into generic symbolic representations, thereby efficiently capturing more generalized reasoning knowledge. We conduct extensive experiments on more than ten datasets encompassing conventional reasoning tasks like arithmetic, symbolic, and logical reasoning, and the more complex interactive reasoning tasks like theory-of-mind reasoning. Experimental results demonstrate that Meta-Reasoning significantly enhances in-context reasoning accuracy, learning efficiency, out-of-domain generalization, and output stability compared to the Chain-of-Thought technique. Code and data are publicly available at: ."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 3.
    Evidence: "existing methods mainly rely on syntactically mapping natural languages to complete formal languages like Python and SQL. Those methods require that reasoning tasks be convertible into programs, which cater to the computer execution mindset and deviate from human reasoning habits."
    
    Output Explanation: This abstract mentions few limitations but not in great detail or as the main focus.   
    
    **Example Output 4:**
    Title: Fairness in Large Language Models: A Taxonomic Survey
    Paper: "Large Language Models (LLMs) have demonstrated remarkable success across various domains. However, despite their promising performance in numerous real-world applications, most of these algorithms lack fairness considerations. Consequently, they may lead to discriminatory outcomes against certain communities, particularly marginalized populations, prompting extensive study in fair LLMs. On the other hand, fairness in LLMs, in contrast to fairness in traditional machine learning, entails exclusive backgrounds, taxonomies, and fulfillment techniques. To this end, this survey presents a comprehensive overview of recent advances in the existing literature concerning fair LLMs. Specifically, a brief introduction to LLMs is provided, followed by an analysis of factors contributing to bias in LLMs. Additionally, the concept of fairness in LLMs is discussed categorically, summarizing metrics for evaluating bias in LLMs and existing algorithms for promoting fairness. Furthermore, resources for evaluating bias in LLMs, including toolkits and datasets, are summarized. Finally, existing research challenges and open questions are discussed."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 4.
    Evidence: "most of these algorithms lack fairness considerations. Consequently, they may lead to discriminatory outcomes against certain communities, particularly marginalized populations," and "an analysis of factors contributing to bias in LLMs. Additionally, the concept of fairness in LLMs is discussed categorically, summarizing metrics for evaluating bias in LLMs and existing algorithms for promoting fairness. Furthermore, resources for evaluating bias in LLMs, including toolkits and datasets, are summarized."
    
    Output Explanation: The paper mentions several limitations related to fairness and bias in LLMs. It discusses these limitations in detail and they are significant, but are discussed alongside other topic.
    
    **Example Output 5:**
    Title: Lost in the Middle: How Language Models Use Long Contexts
    Paper: "While recent language models have the ability to take long contexts as input, relatively little is known about how well they use longer context. We analyze the performance of language models on two tasks that require identifying relevant information in their input contexts: multi-document question answering and key-value retrieval. We find that performance can degrade significantly when changing the position of relevant information, indicating that current language models do not robustly make use of information in long input contexts. In particular, we observe that performance is often highest when relevant information occurs at the beginning or end of the input context, and significantly degrades when models must access relevant information in the middle of long contexts, even for explicitly long-context models. Our analysis provides a better understanding of how language models use their input context and provides new evaluation protocols for future long-context language models."
    Does it talk about LLMs: Yes.
    Rate Limitations of LLMs: 5.
    Evidence: "We find that performance can degrade significantly when changing the position of relevant information, indicating that current language models do not robustly make use of information in long input contexts."
    
    Output Explanation: The entire abstract focuses on the limitations and challenges associated with LLMs' ability to handle long contexts. The whole abstract represents a detailed discussion of a critical limitation

    Please answer in the following format by providing the rating and a brief evidence for each abstract. Please do not give the respective Explanations, only the evidence found in the abstract. 

    Answer in the following way:
    Does it talk about LLMs: [yes/no].
    Rate Limitations of LLMs: [1-5].
    Evidence: [the evidence text in the abstract or title].

    Title: {title}
    Paper: {summary}
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-2024-05-13",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.5
        )
        return response.choices[0]['message']['content'].strip()
    except openai.error.RateLimitError as e:
        print(f"Rate limit error: {e}")
        raise
    except Exception as e:
        print(f"Error during API call: {e}")
        return None

def read_papers_from_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            papers = json.load(file)
        print(f"Successfully read {len(papers)} papers from the JSON file.")
        return papers
    except Exception as e:
        print(f"Error reading JSON file: {e}")
        return []

def get_last_processed_index():
    if os.path.exists(progress_file):
        try:
            with open(progress_file, 'r') as file:
                return int(file.read().strip())
        except Exception as e:
            print(f"Error reading progress file: {e}")
            return -1
    return -1

def save_progress(index):
    try:
        with open(progress_file, 'w') as file:
            file.write(str(index))
        print(f"Progress saved at index: {index}")
    except Exception as e:
        print(f"Error saving progress: {e}")
file_path = 'arXiv_classification_yes_no_results/arXiv_01Feb_to_31Mar_2024_LLM_limitations_part5.json'
csv_file_path = 'arXiv_limitation_ratings/Feb_Mar_2024_ratings_part5.csv'
last_index = get_last_processed_index()
print(f"Last processed index: {last_index}")

papers = read_papers_from_json(file_path)

if not papers:
    print("No papers to process. Exiting.")
    exit()
if last_index >= len(papers):
    last_index = -1

with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    if last_index == -1:
        writer.writerow(['Title', 'Talks about LLMs', 'Rate', 'Evidence', 'Published'])
        print("Header written to CSV file.")

    for index, paper in enumerate(tqdm(papers, desc="Processing papers"), start=0):
        if index <= last_index:
            continue

        title = paper.get('title', 'No Title')
        summary = paper.get('summary', 'No Summary')
        published = paper.get('published', 'No Date')  
        
        max_retries = 5
        retries = 0
        delay = 2
        while retries < max_retries:
            try:
                evaluation_result = evaluate_paper_few_shot(title, summary)
                
                if evaluation_result is None:
                    print(f"Skipping paper at index {index} due to API call failure.")
                    break
                lines = evaluation_result.split('\n')
                talks_about_llms = ''
                rate = ''
                evidence = ''
                
                for line in lines:
                    if line.startswith('Does it talk about LLMs:'):
                        talks_about_llms = line.split(':')[1].strip().strip('[]')
                    elif line.startswith('Rate Limitations of LLMs:'):
                        rate = line.split(':')[1].strip().strip('[]')
                    elif line.startswith('Evidence:'):
                        evidence = line.split(':')[1].strip().strip('[]')
                writer.writerow([title, talks_about_llms, rate, evidence, published])
                print(f"Title: {title}\nEvaluation Result: {evaluation_result}\n")

                save_progress(index)

                time.sleep(1) 
                break
            except openai.error.RateLimitError:
                retries += 1
                print(f"Rate limit error. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2 
                continue
            except Exception as e:
                print(f"An error occurred at index {index}: {e}")
                break

print("Processing complete.")


Error reading progress file: invalid literal for int() with base 10: ''
Last processed index: -1
Successfully read 511 papers from the JSON file.
Header written to CSV file.


Processing papers:   0%|                                                                       | 0/511 [00:00<?, ?it/s]

Title: Generalization in Healthcare AI: Evaluation of a Clinical Large Language Model
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 5.
Evidence: "the potential of these models importantly depends on their ability to generalize effectively across clinical environments and populations, a challenge often underestimated in early development." and "We found poorer generalization particularly in hospitals with fewer samples, among patients with government and unspecified insurance, the elderly, and those with high comorbidities."

Progress saved at index: 0


Processing papers:   0%|                                                               | 1/511 [00:02<23:50,  2.80s/it]

Title: Open-Vocabulary Segmentation with Unpaired Mask-Text Supervision
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 1.
Evidence: "we advocate using the large vision-language model (LVLM) to refine text descriptions and devise a multi-scale ensemble to stablise the matching between masks and entities."

Progress saved at index: 1


...

Title: Towards scalable robotic intervention of children with Autism Spectrum Disorder using LLMs
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 1.
Evidence: "This communication is meant to teach perspective-taking using text generated using a Large Language Model (LLM) pipeline."

Progress saved at index: 509


Processing papers: 100%|████████████████████████████████████████████████████████████▉| 510/511 [24:40<00:02,  2.60s/it]

Title: Efficient Non-Parametric Uncertainty Quantification for Black-Box Large Language Models and Decision Planning
Evaluation Result: Does it talk about LLMs: Yes.
Rate Limitations of LLMs: 3.
Evidence: "This paper focuses on decision planning with uncertainty estimation to address the hallucination problem in language models."

Progress saved at index: 510


Processing papers: 100%|█████████████████████████████████████████████████████████████| 511/511 [24:42<00:00,  2.90s/it]

Processing complete.


### Calculating the Macro-F1 Score 

#### Simplifying Evaluation Metrics for Large Language Model Ratings
##### Objective
The aim is to simplify the evaluation of ratings for papers discussing large language models (LLMs) by grouping the original five rating classes into three distinct classes. This approach can help simplify the evaluation and make it more robust, especially when the distinctions between certain ratings (like 2 and 3) are not very clear.

Rationale
The original five rating classes are as follows:

##### Rating 1: Papers that do not talk about LLMs or do not mention any limitations of LLMs.
##### Rating 2: Papers that mention one limitation of LLMs very briefly.
##### Rating 3: Papers that mention limitations of LLMs, but not as the main focus. These limitations are discussed superficially or as secondary points.
##### Rating 4: Papers that provide multiple significant limitations of LLMs. The limitations are discussed in detail alongside other topics.
##### Rating 5: Papers where the entire abstract or most sentences focus on the limitations and challenges of LLMs, discussing them in detail with strong wording indicating serious issues.
Given this, we propose the following grouping into three classes:

##### Class 0: Papers that should be rated as 1. These papers do not talk about LLMs or do not mention any limitations of LLMs.
##### Class 1: Papers that should be rated as 2 or 3. These papers mention limitations of LLMs but not in detail and not as the main focus. The limitations are secondary points in the discussion.
##### Class 2: Papers that should be rated as 4 or 5. These papers primarily focus on the limitations of LLMs, discussing them in detail throughout the abstract.
Implementation
To implement this grouping, we convert the ratings into three classes using a predefined mapping and then calculate the macro F1 score for each prompting technique compared to the human-annotated ground truth. This ensures the evaluation metrics are independent and accurately reflect the performance of each technique.

In [3]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import os

file_path = os.path.expanduser('~/Desktop/model_vs_ground_truth_comparison.xlsx')
data = pd.read_excel(file_path)
data.columns = data.columns.str.strip()

data['Ground Truth'] = pd.to_numeric(data['Ground Truth'], errors='coerce')
data['Baseline Rate'] = pd.to_numeric(data['Baseline Rate'], errors='coerce')
data['MyPrompt Rate'] = pd.to_numeric(data['MyPrompt Rate'], errors='coerce')
data['MySimplerPrompt'] = pd.to_numeric(data['MySimplerPrompt'], errors='coerce')
data['FewShot'] = pd.to_numeric(data['FewShot'], errors='coerce')
data['FewShotExplanations'] = pd.to_numeric(data['FewShotExplanations'], errors='coerce')

data = data.dropna(subset=['Ground Truth', 'Baseline Rate', 'MyPrompt Rate', 'MySimplerPrompt', 'FewShot', 'FewShotExplanations'])

def convert_to_three_classes(rating):
    if rating == 1:
        return 0
    elif rating in [2, 3]:
        return 1
    elif rating in [4, 5]:
        return 2

data['Ground Truth 3-Class'] = data['Ground Truth'].apply(convert_to_three_classes)
data['Baseline Rate 3-Class'] = data['Baseline Rate'].apply(convert_to_three_classes)
data['MyPrompt Rate 3-Class'] = data['MyPrompt Rate'].apply(convert_to_three_classes)
data['MySimplerPrompt 3-Class'] = data['MySimplerPrompt'].apply(convert_to_three_classes)
data['FewShot 3-Class'] = data['FewShot'].apply(convert_to_three_classes)
data['FewShotExplanations 3-Class'] = data['FewShotExplanations'].apply(convert_to_three_classes)

ground_truth_3_class = data['Ground Truth 3-Class'].astype(int).tolist()

baseline_rate_3_class = data['Baseline Rate 3-Class'].astype(int).tolist()
my_prompt_rate_3_class = data['MyPrompt Rate 3-Class'].astype(int).tolist()
my_simpler_prompt_3_class = data['MySimplerPrompt 3-Class'].astype(int).tolist()
few_shot_3_class = data['FewShot 3-Class'].astype(int).tolist()
few_shot_explanations_3_class = data['FewShotExplanations 3-Class'].astype(int).tolist()

def calculate_metrics(ground_truth, predictions):
    metrics = {
        'accuracy': accuracy_score(ground_truth, predictions),
        'precision': precision_score(ground_truth, predictions, average='macro'),
        'recall': recall_score(ground_truth, predictions, average='macro'),
        'f1': f1_score(ground_truth, predictions, average='macro')
    }
    return metrics

metrics_baseline = calculate_metrics(ground_truth_3_class, baseline_rate_3_class)
metrics_my_prompt = calculate_metrics(ground_truth_3_class, my_prompt_rate_3_class)
metrics_my_simpler_prompt = calculate_metrics(ground_truth_3_class, my_simpler_prompt_3_class)
metrics_few_shot = calculate_metrics(ground_truth_3_class, few_shot_3_class)
metrics_few_shot_explanations = calculate_metrics(ground_truth_3_class, few_shot_explanations_3_class)

def print_metrics(metrics, model_name):
    print(f"{model_name} Metrics:")
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1 Score: {metrics['f1']:.4f}\n")

print_metrics(metrics_baseline, "Prompt1: Standard Baseline Evaluation Prompt")
print_metrics(metrics_my_prompt, "Prompt2: Enhanced Instructional Detail Prompt")
print_metrics(metrics_my_simpler_prompt, "Prompt3: Categorical Instruction Grouping Prompt")
print_metrics(metrics_few_shot, "Prompt4: Selective Few-Shot Prompting Technique")
print_metrics(metrics_few_shot_explanations, "Prompt5: Detailed Explanatory Few-Shot Prompting")


Prompt1: Standard Baseline Evaluation Prompt Metrics:
  Accuracy: 0.5629
  Precision: 0.5843
  Recall: 0.6053
  F1 Score: 0.5522

Prompt2: Enhanced Instructional Detail Prompt Metrics:
  Accuracy: 0.6647
  Precision: 0.6468
  Recall: 0.6577
  F1 Score: 0.6453

Prompt3: Categorical Instruction Grouping Prompt Metrics:
  Accuracy: 0.6946
  Precision: 0.6601
  Recall: 0.6767
  F1 Score: 0.6629

Prompt4: Selective Few-Shot Prompting Technique Metrics:
  Accuracy: 0.7545
  Precision: 0.7219
  Recall: 0.6857
  F1 Score: 0.6982

Prompt5: Detailed Explanatory Few-Shot Prompting Metrics:
  Accuracy: 0.7904
  Precision: 0.7452
  Recall: 0.7384
  F1 Score: 0.7335

